### Imports

In [ ]:
# Imports
import random
from itertools import combinations, cycle
import math
import numpy as np
from copy import deepcopy

# Canvas
!pip install -q ipycanvas==0.11

from google.colab import output
output.enable_custom_widget_manager()

from ipycanvas import Canvas, MultiCanvas, hold_canvas


### Graph

In [ ]:
class Graph:
    """Graph containing node number, edges and degrees."""
    def __init__(self, node_number):
        self.node_number = node_number      # 0 is at the top, counting clokcwise
        self.edges = []     # [[1, 2], [0, 5]]  # (x, y) : x < y
        self.degrees = np.zeros(node_number)     # Degree for each node


    def add_edge(self, edge):
        """Add an edge to the network (if not already there)."""
        if edge not in self.edges and edge[0] != edge[1]:
            self.edges.append(sorted(edge))
            for node_idx in edge:
                self.degrees[node_idx] += 1


    def add_edges(self, edges):
        """Adds multiple edges to the network"""
        for edge in edges:
            self.add_edge(edge)


    # Barabasi-Albert
    def add_node(self, m):
        """Adds a node to the network with m edges, to nodes with degree weights probability"""
        self.node_number += 1
        self.degrees = np.append(self.degrees, 0)
        selected_nodes = np.random.choice(range(self.node_number - 1), m, p=self.degrees[:-1]/sum(self.degrees[:-1]), replace=False)  # Select nodes
        for node in selected_nodes:
            self.add_edge([self.node_number - 1, node])


    def add_nodes(self, n, m):
        """Adds n nodes to the network with degree m"""
        for _ in range(n):
            self.add_node(m)


    # Erdos-Renyi
    def add_random_edges(self, p):
        """Adds every edge with chance p"""
        for edge in combinations(range(self.node_number), 2):
            if edge not in self.edges:  # No duplicate edges
                if random.random() < p:
                    self.edges.append(edge)
    

    # Watts-Strogatz
    def connect_k_neighbour(self, k):
        """Connects the k neighbour of each node"""
        nodes_list = cycle(range(self.node_number))
        current_node = next(nodes_list)     # Starter node : 0
        current_connections = [next(nodes_list) for _ in range(int(k / 2))] # Starter connection : [1, 2]
        for _ in range(self.node_number):
            for c in current_connections:   # Connect connections
                self.add_edge([current_node, c])
            current_node = current_connections.pop(0)   # Next connections
            current_connections.append(next(nodes_list))


    def edit_edges(self, p):
        """Edits edge connections with p probability"""
        for edge in self.edges:
            if random.random() <= p:
                if random.random() <= 0.5:    # First node stays
                    possible_connections = []    #jobb lenne folyamatosan tarolni minden csucshoz a hozza tartozo csucsparokat
                    for i in range(self.node_number):
                        if sorted([edge[0], i]) not in self.edges and i != edge[0]:
                            possible_connections.append(i)
                    new_node = random.choice(possible_connections)
                    self.degrees[edge[1]] -= 1
                    self.degrees[new_node] += 1
                    edge[1] = new_node
                else:                       # Second node stays
                    possible_connections = []    #jobb lenne folyamatosan tarolni minden csucshoz a hozza tartozo csucsparokat
                    for i in range(self.node_number):
                        if sorted([i, edge[1]]) not in self.edges and i != edge[1]:
                            possible_connections.append(i)
                    new_node = random.choice(possible_connections)
                    self.degrees[edge[0]] -= 1
                    self.degrees[new_node] += 1
                    edge[0] = new_node
            pass


    # Draw
    def draw(self):
        """Draws the graph"""
        size = 500
        canvas = MultiCanvas(4, width = size, height = size)

        def perform_drawings(node_number, edges, size): 
            with hold_canvas(canvas):
                # Background
                canvas[0].fill_style = "white"
                canvas[0].fill_rect(0, 0, size, size)

                # Node coords
                graph_radius = size * 0.3

                x_coords = []
                y_coords = []
                for i in range(node_number):
                    radian = ((math.pi * 2) / node_number) * i
                    x_coords.append(size / 2 + math.sin(radian) * graph_radius)
                    y_coords.append(size / 2 - math.cos(radian) * graph_radius)
                
                # Nodes (Circles)
                circle_radius = size / 50

                canvas[2].line_width = 3
                canvas[2].fill_styled_circles(x_coords, y_coords, circle_radius, color=np.array([(255, 255, 255)]*node_number))    # White inside
                canvas[2].stroke_styled_circles(x_coords, y_coords, circle_radius, color=np.array([(0, 0, 0)]*node_number))        # Black outside

                # Edges
                if len(edges) != 0:
                    points = [((x_coords[edge[0]], y_coords[edge[0]]), (x_coords[edge[1]], y_coords[edge[1]])) for edge in edges]
                    canvas[1].stroke_line_segments(points, points_per_line_segment=None)

        args = deepcopy([self.node_number, self.edges, size])
        f = lambda : perform_drawings(*args)
        canvas.on_client_ready(f)
        return canvas


### Main

In [ ]:
def main():
    # Barabasi-Albert
    node_number = 3         # Starter nodes
    m = 2                   # Attachment node number
    graph_BA = Graph(node_number)
    graph_BA.add_edges([[0, 2], [0, 1]])        # Starter edges
    graph_BA.add_nodes(12, m)       # Bonus nodes
    print("Barabasi-Albert")
    display(graph_BA.draw())

    # Erdos-Renyi
    node_number = 15        # Node number
    p = 0.3                 # Chances
    graph_ER = Graph(node_number)
    graph_ER.add_random_edges(p)
    print("Erdos-Renyi")
    display(graph_ER.draw())
    
    # Watts-Strogatz
    node_number = 15        # Node number
    k = 4                   # Neighbour number
    p = 0.2                 # Chances
    graph_WS = Graph(node_number)
    graph_WS.connect_k_neighbour(k)
    graph_WS.edit_edges(p)
    print("Watts-Strogatz")
    display(graph_WS.draw())


### Result

In [ ]:
if __name__ == "__main__":
    main()
    

Barabasi-Albert


MultiCanvas(width=500)

Erdos-Renyi


MultiCanvas(width=500)

Watts-Strogatz


MultiCanvas(width=500)